<a href="https://colab.research.google.com/github/marcogemaque/calculadora-aluguel-turma-set-20/blob/main/1-web-scraping/Awari_DS_Rent_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re

In [34]:
#Let's create a function to retrieve all the rent values
bairros = ['badu','barreto','baldeador','boa-viagem','camboinhas','centro','cubango','cantagalo','charitas',
           'engenho-do-mato','engenhoca','fonseca','fatima','gragoata','icarai','inga','itaipu',
           'ilha-da-conceicao','itacoatiara','jacare','jardim-fluminense','jardim-icarai',
           'largo-do-narradas','largo-da-batalha','maceio','maria-paula','maravista','matapaca',
           'pendotiba',"ponta-dareia",'piratininga','pe-pequeno','rio-do-ouro','santa-rosa',
           'serra-grande','sao-lourenco','santana','sao-domingos','sape','sao-francisco','tenente-jardim',
           'vital-brazil','varzea-das-mocas'] #List created to search in neighborhoods
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', 
           '_Desde_241', '_Desde_289']

In [35]:
def get_rents(x, hood, soup):
    #Function created to fetch rent, m², area and state and other important info
    tag_rent = soup.find_all('span',{"class":"price-tag-fraction"})
    list_rent = [r.text for r in tag_rent] #Create rent list
    tag_street = soup.find_all('h2',{"class":"ui-search-item__title ui-search-item__group__element"})
    list_street = [r.text for r in tag_street] #Create street rent
    tag_area = soup.find_all('ul',{"class":'ui-search-card-attributes'})
    list_area = [r.text for r in tag_area] #Create area list

    df = pd.DataFrame(
        {'Rent':list_rent,
         'Street':list_street,
         'Area':list_area,
         'Hood':hood})
    
    df['Total-Area'] = df['Area'].str.split("[m]+")
    df['Total-Area'] = df['Total-Area'].str[0]
    df['Rooms'] = df['Area'].str.split("[construídos]+").str[1]
    df['Rooms'] = df['Rooms'].str[0]
    #df = df.drop(['Area'],axis=1) #Still not necessary
    df['Rooms'] = df['Rooms'].fillna(0)
    df['Rooms'] = pd.to_numeric(df['Rooms'])
    df['Rooms'] =df['Rooms'].replace(0,df.loc[df['Rooms'] > 0, 'Rooms'].mean())
    df['Total-Area'] = df['Total-Area'].replace("\,","",regex=True)
    df['Total-Area'] = pd.to_numeric(df['Total-Area'])
    df['Rent'] = df['Rent'].replace("\.","",regex=True)
    df['Rent'] = pd.to_numeric(df['Rent'])
    return df

In [36]:
df = pd.DataFrame(columns=["Rent","Street","Area","Hood","Total-Area","Rooms"])
url = 'https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi/'
for bairro in bairros:
    for pag in paginas:
        urls = url + "/{}/{}".format(bairro,pag)
        page = requests.get(urls)
        soup = BeautifulSoup(page.content,'html.parser')
        print('Currently scraping: {}'.format(urls))
        page = requests.get(urls)
        try:
            df_int = get_rents(page,bairro,soup)
            print(df_int.head())
            list_dfs = [df, df_int]
            df = pd.concat(list_dfs)
            sleep(1)
        except:
            print('exception')
            sleep(1)
            pass

Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/
   Rent                    Street  ... Total-Area     Rooms
0  1650  Estrada Caetano Monteiro  ...         70  2.000000
1  3500  Estrada Caetano Monteiro  ...         90  2.894737
2  1300              391 Bloco 06  ...         82  2.000000
3  1000  Estrada Caetano Monteiro  ...         60  2.000000
4  4500                            ...        400  4.000000

[5 rows x 6 columns]
Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/_Desde_49
exception
Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/_Desde_97
exception
Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/_Desde_145
exception
Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de-janeiro/niteroi//badu/_Desde_193
exception
Currently scraping: https://imoveis.mercadolivre.com.br/aluguel/rio-de

In [37]:
df_out = df.copy()
df_out.to_csv("consolidado.csv")

In [38]:
df_out.head()

,Rent,Street,Area,Hood,Total-Area,Rooms
0,1650,Estrada Caetano Monteiro,70 m² construídos2 quartos,badu,70,2.000000
1,3500,Estrada Caetano Monteiro,90 m² construídos,badu,90,2.894737
2,1300,391 Bloco 06,82 m² construídos2 quartos,badu,82,2.000000
3,1000,Estrada Caetano Monteiro,60 m² construídos2 quartos,badu,60,2.000000
4,4500,,400 m² construídos4 quartos,badu,400,4.000000


In [39]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1241 entries, 0 to 2
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Rent        1241 non-null   object 
 1   Street      1241 non-null   object 
 2   Area        1241 non-null   object 
 3   Hood        1241 non-null   object 
 4   Total-Area  1241 non-null   object 
 5   Rooms       1241 non-null   float64
dtypes: float64(1), object(5)
memory usage: 67.9+ KB


In [41]:
df_out.describe()

,Rooms
count,1241.000000
mean,2.311673
std,0.915239
min,1.000000
25%,2.000000
50%,2.000000
75%,3.000000
max,7.000000


In [42]:
df_out = df_out.sort_values(by=['Rent'])

In [43]:
df_out

,Rent,Street,Area,Hood,Total-Area,Rooms
42,150,Amaral Peixoto 347,2 m² construídos,centro,2,1.470588
23,300,,30 m² construídos,centro,30,1.916667
27,350,,46 m² construídos,centro,46,1.916667
34,350,,56 m² construídos,centro,56,1.470588
44,390,Av. Ernani Amaral Peixoto 460 405,32 m² construídos,centro,32,1.470588
...,...,...,...,...,...,...
22,60000,"Alameda São Boa Ventura, 121",750 m² construídos,fonseca,750,2.317073
30,70000,,"3,000 m² construídos",centro,3000,1.655172
41,100000,Rua Coronel Gomes Machado 173,16 m² construídos,centro,16,1.470588
10,2000000,,146 m² construídos4 quartos,camboinhas,146,4.000000
